In [ ]:
import psycopg2
import json

# Database connection parameters
conn_params = {
    'dbname': 'your_database',
    'user': 'your_user',
    'password': 'your_password',
    'host': 'your_host',
    'port': 'your_port'
}

# Connect to PostgreSQL
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

# Create the graph
cur.execute("SELECT create_graph('pg2_case_graph');")
conn.commit()

# Insert nodes for each case
cur.execute("SELECT id, data FROM public.cases_playground;")
cases = cur.fetchall()

for case in cases:
    case_id, case_data = case
    cur.execute("""
        SELECT * FROM cypher('pg2_case_graph', %s) as (c agtype);
    """, (f"""
        CREATE (c:Case {{id: '{case_id}', data: '{json.dumps(case_data)}'}})
    """,))
    conn.commit()

# Insert edges based on citations
cur.execute("SELECT id, data FROM public.cases;")
cases = cur.fetchall()

for case in cases:
    case_id, case_data = case
    citations = json.loads(case_data).get('citations', [])
    for citation in citations:
        cited_case_id = citation.get('case_id')
        cur.execute("""
            SELECT * FROM cypher('pg2_case_graph', %s) as (c1 agtype, c2 agtype);
        """, (f"""
            MATCH (c1:Case {{id: '{case_id}'}}), (c2:Case {{id: '{cited_case_id}'}})
            CREATE (c1)-[:CITES]->(c2)
        """,))
        conn.commit()

# Close the connection
cur.close()
conn.close()
